<a href="https://colab.research.google.com/github/yujinc129-oss/Drama_data_project/blob/main/drama_machine(2)_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn. ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer


#2차 전처리

In [2]:
drama = pd.read_csv('https://raw.githubusercontent.com/yujinc129-oss/Drama_data_project/refs/heads/main/dataset/data_drama_set')

In [3]:
drama

,Unnamed: 0,actor,drama,role,gender,Genres,Aired On,Original Network,score_weighted,age
0,0,junghaein,snowdrop,Main Role,Male,"['romance', 'society', 'drama']","['saturday', ' sunday']",['JTBC'],8.20,34
1,1,junghaein,d.p.,Main Role,Male,"['action', 'hist_war', 'drama']",['friday'],['NETFLIX'],8.43,34
2,2,junghaein,apieceofyourmind,Main Role,Male,"['romance', 'drama']","['monday', ' tuesday']",['TVN'],7.97,33
3,3,junghaein,onespringnight,Main Role,Male,"['romance', 'drama']","['thursday', ' wednesday']","['MBC', ' NETFLIX']",8.17,32
4,4,junghaein,somethingintherain,Main Role,Male,"['romance', 'drama']","['friday', ' saturday']","['JTBC', ' NETFLIX']",7.92,31
...,...,...,...,...,...,...,...,...,...,...
2212,2212,kimjongkook,theproducers,Support Role,Male,"['romance', 'drama', 'comedy']","['friday', ' saturday']",['KBS2'],7.64,40
2213,2213,jungyoonhak,myunfortunateboyfriend,Main Role,Male,"['romance', 'drama']","['friday', ' saturday']",['MBC DRAMANET'],7.54,32
2214,2214,jungjoonyoung,thelover,Main Role,Male,"['romance', 'drama', 'comedy']",['thursday'],['MNET'],7.80,27
2215,2215,kimjoon,cityofthesun,Main Role,Male,"['thriller', 'society', 'drama', 'romance']","['friday', ' saturday']",['MBC DRAMANET'],7.70,32


In [4]:
drama = drama.drop(columns=['Unnamed: 0'])

In [5]:
drama

,actor,drama,role,gender,Genres,Aired On,Original Network,score_weighted,age
0,junghaein,snowdrop,Main Role,Male,"['romance', 'society', 'drama']","['saturday', ' sunday']",['JTBC'],8.20,34
1,junghaein,d.p.,Main Role,Male,"['action', 'hist_war', 'drama']",['friday'],['NETFLIX'],8.43,34
2,junghaein,apieceofyourmind,Main Role,Male,"['romance', 'drama']","['monday', ' tuesday']",['TVN'],7.97,33
3,junghaein,onespringnight,Main Role,Male,"['romance', 'drama']","['thursday', ' wednesday']","['MBC', ' NETFLIX']",8.17,32
4,junghaein,somethingintherain,Main Role,Male,"['romance', 'drama']","['friday', ' saturday']","['JTBC', ' NETFLIX']",7.92,31
...,...,...,...,...,...,...,...,...,...
2212,kimjongkook,theproducers,Support Role,Male,"['romance', 'drama', 'comedy']","['friday', ' saturday']",['KBS2'],7.64,40
2213,jungyoonhak,myunfortunateboyfriend,Main Role,Male,"['romance', 'drama']","['friday', ' saturday']",['MBC DRAMANET'],7.54,32
2214,jungjoonyoung,thelover,Main Role,Male,"['romance', 'drama', 'comedy']",['thursday'],['MNET'],7.80,27
2215,kimjoon,cityofthesun,Main Role,Male,"['thriller', 'society', 'drama', 'romance']","['friday', ' saturday']",['MBC DRAMANET'],7.70,32


In [6]:
drama = drama.rename(columns={'Genres':'genres','Aired On':'day','Original Network' :'network','score_weighted':'score'})

In [7]:
import ast
# 1. ast.literal_eval로 문자열을 실제 리스트로 변환
# 2. 리스트 내포로 각 항목의 공백 제거
drama['day'] = drama['day'].apply(lambda x: [day.strip() for day in ast.literal_eval(x)])


In [8]:
drama['genres'] = drama['genres'].apply(lambda x: [day.strip() for day in ast.literal_eval(x)])
drama['network'] = drama['network'].apply(lambda x: [day.strip() for day in ast.literal_eval(x)])
drama['day'] = drama['day'].apply(lambda day_list: [day.strip() for day in day_list])

In [9]:
drama

,actor,drama,role,gender,genres,day,network,score,age
0,junghaein,snowdrop,Main Role,Male,"[romance, society, drama]","[saturday, sunday]",[JTBC],8.20,34
1,junghaein,d.p.,Main Role,Male,"[action, hist_war, drama]",[friday],[NETFLIX],8.43,34
2,junghaein,apieceofyourmind,Main Role,Male,"[romance, drama]","[monday, tuesday]",[TVN],7.97,33
3,junghaein,onespringnight,Main Role,Male,"[romance, drama]","[thursday, wednesday]","[MBC, NETFLIX]",8.17,32
4,junghaein,somethingintherain,Main Role,Male,"[romance, drama]","[friday, saturday]","[JTBC, NETFLIX]",7.92,31
...,...,...,...,...,...,...,...,...,...
2212,kimjongkook,theproducers,Support Role,Male,"[romance, drama, comedy]","[friday, saturday]",[KBS2],7.64,40
2213,jungyoonhak,myunfortunateboyfriend,Main Role,Male,"[romance, drama]","[friday, saturday]",[MBC DRAMANET],7.54,32
2214,jungjoonyoung,thelover,Main Role,Male,"[romance, drama, comedy]",[thursday],[MNET],7.80,27
2215,kimjoon,cityofthesun,Main Role,Male,"[thriller, society, drama, romance]","[friday, saturday]",[MBC DRAMANET],7.70,32


## 장르/플랫폼 카테고리 정리

In [10]:
gen_list = drama.genres.sum()
gen_list.sort()
gen_np = pd.DataFrame(gen_list)
gen_np.value_counts()

,count
0,
romance,1517
drama,1464
comedy,962
thriller,889
society,600
sf,461
action,273
hist_war,202
youth,128


In [11]:
#장르 변수 수정
def to_etc(drama_gen_list):
  etc = ['action','hist_war','youth','music','food','sports','etc']  # 총 10개 장르
  new = set()

  for x in drama_gen_list:
    if x in etc:
      new.add('etc')
    else:
      new.add(x)
  return new

test_1 = ['society', 'romance', 'drama', 'food']

print(to_etc(test_1))

{'society', 'etc', 'drama', 'romance'}


In [12]:
for x in range(len(drama.genres)):
  drama.genres.loc[x] = to_etc(drama.genres.loc[x])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  drama.genres.loc[x] = to_etc(drama.genres.loc[x])
/tmp/ipython-input-2058781471.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drama.genres.loc[x] = to_etc(drama.genres.loc[x])
/tmp/ipython-input-2058781471.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical exam

In [13]:
#네트워크 변수 수정  총 12개 네트워크
net_dict={
          'ETC':['IQIYI', 'VIUTV', 'OKSUSU','BS JAPAN','MNET','DRAMAX','TV CHOSUN', 'CHANNEL A', 'MBN',
                 'NAVER TV CAST', 'DAUM KAKAO TV', 'VLIVE','VIKI' ],
          'KBS' : ['KBS1','KBS2'],
          'MBC': ['MBC','MBC DRAMANET', 'MBC EVERY1'],
          'SBS' : ['SBS','SBS PLUS']}

In [14]:
network = pd.DataFrame(drama.network.sum())
network.value_counts()

,count
0,
TVN,466
KBS2,440
MBC,340
SBS,331
NETFLIX,290
JTBC,230
VIKI,146
OCN,123
KBS1,61


In [15]:

def net_change(network_list):
  new = set()

  for x in network_list:
    k= False
    for key, value in net_dict.items():
      if x in value:
        new.add(key)
        k=True

    if k==False:
        new.add(x)
  return new


net_change(['NAVER TV CAST', 'NETFLIX'])

{'ETC', 'NETFLIX'}

In [16]:
for x in range(len(drama.network)):
  drama.network[x] = net_change(drama.network[x])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  drama.network[x] = net_change(drama.network[x])
/tmp/ipython-input-738861697.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drama.network[x] = net_change(drama.network[x])
/tmp/ipython-input-738861697.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is 

## 파생변수 추가

In [18]:
#방영연도 컬럼추가

year = pd.read_csv('https://github.com/yujinc129-oss/Drama_data_project/raw/refs/heads/main/dataset/data_for_add')
aired_year = year['start airing']

In [19]:
aired_year = year[['drama_name','start airing']]

In [20]:
drama = pd.merge(drama, aired_year, left_on='drama', right_on='drama_name')

In [21]:
drama= drama.drop(columns='drama_name')

In [22]:
drama

,actor,drama,role,gender,genres,day,network,score,age,start airing
0,junghaein,snowdrop,Main Role,Male,"{society, drama, romance}","[saturday, sunday]",{JTBC},8.20,34,2021
1,junghaein,d.p.,Main Role,Male,"{etc, drama}",[friday],{NETFLIX},8.43,34,2021
2,junghaein,apieceofyourmind,Main Role,Male,"{drama, romance}","[monday, tuesday]",{TVN},7.97,33,2020
3,junghaein,onespringnight,Main Role,Male,"{drama, romance}","[thursday, wednesday]","{MBC, NETFLIX}",8.17,32,2019
4,junghaein,somethingintherain,Main Role,Male,"{drama, romance}","[friday, saturday]","{JTBC, NETFLIX}",7.92,31,2018
...,...,...,...,...,...,...,...,...,...,...
2212,kimjongkook,theproducers,Support Role,Male,"{comedy, drama, romance}","[friday, saturday]",{KBS},7.64,40,2015
2213,jungyoonhak,myunfortunateboyfriend,Main Role,Male,"{drama, romance}","[friday, saturday]",{MBC},7.54,32,2015
2214,jungjoonyoung,thelover,Main Role,Male,"{comedy, drama, romance}",[thursday],{ETC},7.80,27,2015
2215,kimjoon,cityofthesun,Main Role,Male,"{society, drama, romance, thriller}","[friday, saturday]",{MBC},7.70,32,2015


In [23]:
#주연비율 파생변수 생성
drama_cnt = drama.actor.value_counts(sort=False)
main_cnt =drama[drama['role']=='Main Role'].groupby('actor',sort=False,)['drama'].count()
main_cnt = main_cnt.fillna(0)
drama_cnt = drama_cnt.fillna(0)
ratio = main_cnt / drama_cnt
ratio = ratio.replace([np.inf, -np.inf], np.nan).fillna(0)

ratio =ratio.reset_index()

drama = pd.merge(drama, ratio, how='left')

drama.rename(columns={0:'main_ratio'},inplace=True)

drama

,actor,drama,role,gender,genres,day,network,score,age,start airing,main_ratio
0,junghaein,snowdrop,Main Role,Male,"{society, drama, romance}","[saturday, sunday]",{JTBC},8.20,34,2021,0.75
1,junghaein,d.p.,Main Role,Male,"{etc, drama}",[friday],{NETFLIX},8.43,34,2021,0.75
2,junghaein,apieceofyourmind,Main Role,Male,"{drama, romance}","[monday, tuesday]",{TVN},7.97,33,2020,0.75
3,junghaein,onespringnight,Main Role,Male,"{drama, romance}","[thursday, wednesday]","{MBC, NETFLIX}",8.17,32,2019,0.75
4,junghaein,somethingintherain,Main Role,Male,"{drama, romance}","[friday, saturday]","{JTBC, NETFLIX}",7.92,31,2018,0.75
...,...,...,...,...,...,...,...,...,...,...,...
2212,kimjongkook,theproducers,Support Role,Male,"{comedy, drama, romance}","[friday, saturday]",{KBS},7.64,40,2015,0.00
2213,jungyoonhak,myunfortunateboyfriend,Main Role,Male,"{drama, romance}","[friday, saturday]",{MBC},7.54,32,2015,1.00
2214,jungjoonyoung,thelover,Main Role,Male,"{comedy, drama, romance}",[thursday],{ETC},7.80,27,2015,1.00
2215,kimjoon,cityofthesun,Main Role,Male,"{society, drama, romance, thriller}","[friday, saturday]",{MBC},7.70,32,2015,1.00


In [24]:
#플랫폼 등급 컬럼 추가
# 1. 등급 매핑 딕셔너리 정의 (숫자가 작을수록 높은 등급)
tier_map = {'NETFLIX':1, 'TVN':1, 'JTBC':1, 'KBS':2, 'MBC':2, 'SBS':2, 'OCN':2, 'ETC':3}
default_tier = 3

# 2. 가장 높은 등급을 찾아주는 함수 정의
def get_highest_tier(platform_list):
    # 리스트에 있는 각 플랫폼의 등급을 찾음
    # tier_map에 없는 플랫폼은 default_tier로 처리
    tiers = [tier_map.get(p, default_tier) for p in platform_list]

    # 등급 리스트가 비어있으면 기본 등급 반환
    if not tiers:
        return default_tier

    # 가장 작은 숫자(가장 높은 등급)를 반환
    return min(tiers)


# 3. apply 함수를 사용하여 'platform_tier' 컬럼 생성
drama['platform_tier'] = drama['network'].apply(get_highest_tier)

drama

,actor,drama,role,gender,genres,day,network,score,age,start airing,main_ratio,platform_tier
0,junghaein,snowdrop,Main Role,Male,"{society, drama, romance}","[saturday, sunday]",{JTBC},8.20,34,2021,0.75,1
1,junghaein,d.p.,Main Role,Male,"{etc, drama}",[friday],{NETFLIX},8.43,34,2021,0.75,1
2,junghaein,apieceofyourmind,Main Role,Male,"{drama, romance}","[monday, tuesday]",{TVN},7.97,33,2020,0.75,1
3,junghaein,onespringnight,Main Role,Male,"{drama, romance}","[thursday, wednesday]","{MBC, NETFLIX}",8.17,32,2019,0.75,1
4,junghaein,somethingintherain,Main Role,Male,"{drama, romance}","[friday, saturday]","{JTBC, NETFLIX}",7.92,31,2018,0.75,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2212,kimjongkook,theproducers,Support Role,Male,"{comedy, drama, romance}","[friday, saturday]",{KBS},7.64,40,2015,0.00,2
2213,jungyoonhak,myunfortunateboyfriend,Main Role,Male,"{drama, romance}","[friday, saturday]",{MBC},7.54,32,2015,1.00,2
2214,jungjoonyoung,thelover,Main Role,Male,"{comedy, drama, romance}",[thursday],{ETC},7.80,27,2015,1.00,3
2215,kimjoon,cityofthesun,Main Role,Male,"{society, drama, romance, thriller}","[friday, saturday]",{MBC},7.70,32,2015,1.00,2


In [ ]:
#감독 작가 파생변수 생성 -ing
director = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_set/DRAMA_PROJECT/etc/drama/korean_drama.csv')
director[['drama_name','director','screenwriter']]

,drama_name,director,screenwriter
0,Sing My Crush,['So Joon Moon'],NaN
1,D.P. Season 2,NaN,['Kim Bo Tong']
2,Shadow Detective Season 2,['Han Dong Hwa'],"['Song Jung Woo', 'Hwang Seol Hun']"
3,To Be Honest,NaN,NaN
4,Celebrity,['Kim Chul Gyu'],['Kim Yi Young']
...,...,...,...
1747,"Kill Me, Heal Me","['Kim Jin Man', 'Kim Dae Jin']",['Jin Soo Wan']
1748,Enchanting Neighbor,['Park Kyung Ryul'],['Park Hye Ryun']
1749,Iron Lady Cha,"['Jang Joon Ho', 'Oh Hyun Chang']",['Seol Kyung Eun']
1750,The Family Is Coming,['Joo Dong Min'],['Kim Shin Hye']


#인코딩

In [ ]:
mlb = MultiLabelBinarizer()

fmt = mlb.fit_transform(drama['genres'])
gen = pd.DataFrame(fmt, columns=mlb.classes_, index=drama.index)


mlb = MultiLabelBinarizer()

fmt = mlb.fit_transform(drama['day'])
day = pd.DataFrame(fmt, columns=mlb.classes_, index=drama.index)

mlb = MultiLabelBinarizer()

fmt = mlb.fit_transform(drama['network'])
net = pd.DataFrame(fmt, columns=mlb.classes_, index=drama.index)


In [ ]:
net

,ETC,JTBC,KBS,MBC,NETFLIX,OCN,SBS,TVN
0,0,1,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,1
3,0,0,0,1,1,0,0,0
4,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...
2212,0,0,1,0,0,0,0,0
2213,0,0,0,1,0,0,0,0
2214,1,0,0,0,0,0,0,0
2215,0,0,0,1,0,0,0,0


In [ ]:
others = pd.get_dummies(drama[['role','gender']])

In [ ]:
X = pd.concat([drama['age'],gen, day, net, others], axis=1)


In [ ]:
X

,age,comedy,drama,etc,romance,sf,society,thriller,friday,monday,...,KBS,MBC,NETFLIX,OCN,SBS,TVN,role_Main Role,role_Support Role,gender_Female,gender_Male
0,34,0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,True,False,False,True
1,34,0,1,1,0,0,0,0,1,0,...,0,0,1,0,0,0,True,False,False,True
2,33,0,1,0,1,0,0,0,0,1,...,0,0,0,0,0,1,True,False,False,True
3,32,0,1,0,1,0,0,0,0,0,...,0,1,1,0,0,0,True,False,False,True
4,31,0,1,0,1,0,0,0,1,0,...,0,0,1,0,0,0,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2212,40,1,1,0,1,0,0,0,1,0,...,1,0,0,0,0,0,False,True,False,True
2213,32,0,1,0,1,0,0,0,1,0,...,0,1,0,0,0,0,True,False,False,True
2214,27,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,True,False,False,True
2215,32,0,1,0,1,0,1,1,1,0,...,0,1,0,0,0,0,True,False,False,True


In [ ]:
X_data = X.copy()
data_processed = pd.concat([X_data, drama[['start airing','main_ratio','platform_tier']]], axis=1)

In [ ]:
#X변수
X = data_processed
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2217 entries, 0 to 2216
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                2217 non-null   int64  
 1   comedy             2217 non-null   int64  
 2   drama              2217 non-null   int64  
 3   etc                2217 non-null   int64  
 4   romance            2217 non-null   int64  
 5   sf                 2217 non-null   int64  
 6   society            2217 non-null   int64  
 7   thriller           2217 non-null   int64  
 8   friday             2217 non-null   int64  
 9   monday             2217 non-null   int64  
 10  saturday           2217 non-null   int64  
 11  sunday             2217 non-null   int64  
 12  thursday           2217 non-null   int64  
 13  tuesday            2217 non-null   int64  
 14  wednesday          2217 non-null   int64  
 15  ETC                2217 non-null   int64  
 16  JTBC               2217 

In [ ]:
#y 변수
y= drama['score']

#train/test split

In [ ]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42)

#Linear Regression

In [ ]:
linear = LinearRegression()
linear.fit(X_tr,y_tr)

LinearRegression()

In [ ]:
Xpred = linear.predict(X_tr)


In [ ]:
cv = cross_validate(linear, X_tr, y_tr, cv=5, n_jobs=1, return_train_score=True)

In [ ]:
cv

{'fit_time': array([0.00517702, 0.00543714, 0.00463223, 0.00545549, 0.00564265]),
 'score_time': array([0.00250244, 0.00323963, 0.00251102, 0.00297499, 0.0025177 ]),
 'test_score': array([0.103098  , 0.17635233, 0.17950947, 0.18095582, 0.17473131]),
 'train_score': array([0.21652719, 0.1970414 , 0.19764744, 0.19431445, 0.19847372])}

In [ ]:
#결과
print(f'TRAIN_SCORE : {cv["train_score"].mean()}  |  TEST_SCORE : {cv["test_score"].mean()}')
print('MAE :',mean_absolute_error(y_tr, Xpred))
print('MSE :',mean_squared_error(y_tr, Xpred))
print('R2 :',r2_score(y_tr, Xpred))

TRAIN_SCORE : 0.2008008411904557  |  TEST_SCORE : 0.1629293854122472
MAE : 0.19035330515642895
MSE : 0.0646255953265736
R2 : 0.1971937561509629


#Randomforest

In [ ]:
randomf = RandomForestRegressor()

In [ ]:
randomf.fit(X_tr, y_tr)
X_pred = randomf.predict(X_tr)

In [ ]:
scoring_metrics = ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']
cv = cross_validate(randomf, X_tr, y_tr, n_jobs=-1, return_train_score=True, scoring=scoring_metrics )


train_r2_mean = cv['train_r2'].mean()
train_mse_mean = -cv['train_neg_mean_squared_error'].mean() # 음수이므로 -1을 곱해 복원
train_mae_mean = -cv['train_neg_mean_absolute_error'].mean()

test_r2_mean = cv['test_r2'].mean()
test_mse_mean = -cv['test_neg_mean_squared_error'].mean() # 음수이므로 -1을 곱해 복원
test_mae_mean = -cv['test_neg_mean_absolute_error'].mean() # 음수이므로 -1을 곱해 복원
#결과
print(f'TRAIN MAE : {train_mae_mean} / TEST MAE : {test_mae_mean}' )
print(f'TRAIN MSE : {train_mse_mean} / TEST MSE : {test_mse_mean}')
print(f'TRAIN R2 : {train_r2_mean} / TEST R2 : {test_r2_mean}')

TRAIN MAE : 0.04007244649665246 / TEST MAE : 0.11026472429378571
TRAIN MSE : 0.003561889511652078 / TEST MSE : 0.025719779015773648
TRAIN R2 : 0.9557190786196081 / TEST R2 : 0.6816700017525281


##RandomizedSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import scipy

In [ ]:
# #params = {'max_depth' : scipy.stats.randint(100,150),
#           'min_samples_leaf' : scipy.stats.randint(2,10),
#           }

#randcv = RandomizedSearchCV(randomf, params, n_iter= 100, n_jobs=-1, return_train_score=True)

#randcv.fit(X_tr, y_tr)

#GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
16, 2, 400

(16, 2, 400)

In [ ]:
params = { 'n_estimators': [600],  # 이전 최적값으로 고정
    'max_depth': [10, 12, 15], # 이전 최적값(18)보다 의도적으로 낮게 설정
    'min_samples_leaf': [5, 10, 15]                 # 간격을 두고 3개 값 테스트
    #'min_samples_split' : [2, 5, 10],                # 기존 범위에서 일부만 선택
    #'max_features': ['sqrt']
           }

grid = GridSearchCV(randomf, params, n_jobs=-1, cv= 5, return_train_score=True,)
grid.fit(X_tr, y_tr)


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [10, 12, 15],
                         'min_samples_leaf': [5, 10, 15],
                         'n_estimators': [600]},
             return_train_score=True)

In [ ]:
grid.best_estimator_.feature_importances_

array([0.0731736 , 0.05347314, 0.04788873, 0.05480181, 0.0314618 ,
       0.04633975, 0.03460083, 0.04535032, 0.01994989, 0.02259408,
       0.02285641, 0.01956463, 0.01202714, 0.01900614, 0.01290345,
       0.01550084, 0.02171881, 0.02592721, 0.00828639, 0.06249972,
       0.00736152, 0.01437359, 0.05278865, 0.00254145, 0.00245057,
       0.00484932, 0.00493837, 0.14341942, 0.03077925, 0.08657314])

max_depth=np.int64(21), n_estimators=1000   0.5669713893907954

max_depth=np.int64(19), min_samples_leaf=2, n_estimators=1000  0.5347813665813888

In [ ]:
grid.best_score_

np.float64(0.5001116918275621)

In [ ]:
grid.best_params_

{'max_depth': 15, 'min_samples_leaf': 5, 'n_estimators': 600}

In [ ]:
best_model = grid.best_estimator_

# 2. 변수 중요도를 계산합니다.
importances = best_model.feature_importances_

# 3. 컬럼 이름은 학습 데이터(X_tr)에서 바로 가져옵니다.
feature_names = X_tr.columns

# 4. 컬럼 이름과 중요도를 합쳐서 보기 좋게 출력합니다.
feature_importances = pd.Series(importances, index=feature_names).sort_values(ascending=False)

print("--- 변수 중요도 (상위 20개) ---")
print(feature_importances)

--- 변수 중요도 (상위 20개) ---
start airing         0.143419
platform_tier        0.086573
age                  0.073174
NETFLIX              0.062500
etc                  0.054802
comedy               0.053473
TVN                  0.052789
drama                0.047889
sf                   0.046340
thriller             0.045350
society              0.034601
romance              0.031462
main_ratio           0.030779
KBS                  0.025927
saturday             0.022856
monday               0.022594
JTBC                 0.021719
friday               0.019950
sunday               0.019565
tuesday              0.019006
ETC                  0.015501
SBS                  0.014374
wednesday            0.012903
thursday             0.012027
MBC                  0.008286
OCN                  0.007362
gender_Male          0.004938
gender_Female        0.004849
role_Main Role       0.002541
role_Support Role    0.002451
dtype: float64


In [ ]:
score_history = []

results = pd.DataFrame(grid.cv_results_)
results['train-test'] = results['mean_train_score'] - results['mean_test_score']
results_10 = results[['param_max_depth', 'param_min_samples_leaf', 'param_n_estimators',
       'mean_test_score', 'std_test_score','train-test']].sort_values(by=['mean_test_score','train-test'], ascending=[False, True]).head(10)

score_history.extend([[results_10.iloc[0]]])

results_10

,param_max_depth,param_min_samples_leaf,param_n_estimators,mean_test_score,std_test_score,train-test
6,15,5,600,0.500112,0.024039,0.232175
3,12,5,600,0.498929,0.024364,0.231446
0,10,5,600,0.495040,0.023829,0.224353
7,15,10,600,0.389553,0.024493,0.161696
4,12,10,600,0.386550,0.026124,0.164577
1,10,10,600,0.385096,0.022883,0.163482
2,10,15,600,0.331944,0.023129,0.120729
8,15,15,600,0.331307,0.022730,0.121943
5,12,15,600,0.331098,0.024116,0.122311


#XGBOOST

In [ ]:
from xgboost import XGBRegressor

In [ ]:
cv = cross_validate(xg,X_tr, y_tr, n_jobs= -1, return_train_score=True )
print(cv['train_score'].mean())
print(cv['test_score'].mean())

In [ ]:
params = {
    'learning_rate': [0.01, 0.03],
    'max_depth': [5, 6, 7],

    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

grid = GridSearchCV(xg, params, n_jobs=-1, cv= 5, return_train_score=True)
grid.fit(X_tr, y_tr)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 0.8, 0.9],
                         'learning_rate': [0.01, 0.03], 'max_depth': [5, 6, 7],
                         'subsample': [0.7, 0.8, 0.9]},
             return_train_score=True)

In [ ]:
grid.best_params_

{'colsample_bytree': 0.9,
 'learning_rate': 0.03,
 'max_depth': 7,
 'subsample': 0.9}

In [ ]:
grid.best_score_

np.float64(0.6287684728249661)

In [ ]:
results = pd.DataFrame(grid.cv_results_)
results['train-test'] = results['mean_train_score'] - results['mean_test_score']
results_10 = results[['param_colsample_bytree', 'param_learning_rate', 'param_max_depth',
        'param_subsample','mean_train_score', 'mean_test_score','train-test']].sort_values(by=['mean_test_score','train-test'], ascending=[False, True]).head(10)

results_10

,param_colsample_bytree,param_learning_rate,param_max_depth,param_subsample,mean_train_score,mean_test_score,train-test
53,0.9,0.03,7,0.9,0.833449,0.628768,0.204680
35,0.8,0.03,7,0.9,0.826281,0.619616,0.206665
52,0.9,0.03,7,0.8,0.827530,0.600263,0.227267
17,0.7,0.03,7,0.9,0.814468,0.598210,0.216258
34,0.8,0.03,7,0.8,0.815619,0.591775,0.223844
51,0.9,0.03,7,0.7,0.817413,0.585595,0.231819
16,0.7,0.03,7,0.8,0.806276,0.575848,0.230428
33,0.8,0.03,7,0.7,0.808039,0.573298,0.234741
15,0.7,0.03,7,0.7,0.796560,0.556783,0.239777
50,0.9,0.03,6,0.9,0.730165,0.545593,0.184571


In [ ]:
grid.best_params_

{'colsample_bytree': 0.9,
 'learning_rate': 0.03,
 'max_depth': 7,
 'subsample': 0.9}